In [23]:
import os
from datetime import datetime

import gradio as gr
import ipywidgets as widgets
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from RiskMetrics import RiskAnalysis, create_constraint, diversification_constraint
from scipy.optimize import minimize

In [24]:
def objective(w):
    return np.sqrt(np.sum((w - w0) ** 2))


def sum_equal_one(weight):
    return np.sum(weight) - 1

In [25]:
def display_scrollable_df(df, max_height="50vh", max_width="90vw"):
    style = f"""
    <div style="
        display: flex;
        justify-content: center;
        padding: 20px;
    ">
        <div style="
            overflow: auto;
            max-height: {max_height};
            max-width: {max_width};
            width: 100%;
            border: 1px solid #444;
            padding: 10px;
            background-color: #000;
            color: #eee;
            font-family: 'Arial Narrow', Arial, sans-serif;
            box-sizing: border-box;
        ">
            {df.to_html(classes='table', border=0, index=True)}
        </div>
    </div>
    """
    return HTML(style)

In [26]:
def build_constraint(constraint_matrix):

    global data, constraints

    constraint_matrix = pd.DataFrame(data).to_numpy()

    constraints = [{"type": "eq", "fun": sum_equal_one}]

    dico_map = {"=": "eq", "≥": "ineq", "≤": "ineq"}

    try:
        for row in range(constraint_matrix.shape[0]):
            temp = constraint_matrix[row, :]
            ticker = temp[0]

            if ticker not in drop_down_list:
                continue

            sign = temp[1]
            limit = float(temp[2])

            if ticker == "All":
                constraint = diversification_constraint(sign, limit)

            elif ticker in drop_down_list_asset:
                position = np.where(full_matrix.index == ticker)[0][0]
                constraint = create_constraint(sign, limit, position)

            elif ticker in drop_down_list_sector:
                position = np.where(full_matrix.columns == ticker)[0][0]
                if sign == "≤":
                    constraint = [
                        {
                            "type": dico_map[sign],
                            "fun": lambda weights: limit
                            - (weights @ full_matrix_numpy)[position],
                        }
                    ]
                elif sign == "≥":
                    constraint = [
                        {
                            "type": dico_map[sign],
                            "fun": lambda weights: (weights @ full_matrix_numpy)[
                                position
                            ]
                            - limit,
                        }
                    ]
                else:
                    constraint = [
                        {
                            "type": dico_map[sign],
                            "fun": lambda weights: (weights @ full_matrix_numpy)[
                                position
                            ]
                            - limit,
                        }
                    ]

            constraints.extend(constraint)

    except Exception as e:
        print(f"Error in build_constraint: {e}")

    return constraints

In [27]:
def reset_constraints():
    global data, constraints
    data = pd.DataFrame(columns=["Asset", "Sign", "Limit"])
    constraints = []
    return data

In [49]:

def display_app(file):
    
    data_file = file.parse(sheet_name=file.sheet_names)
    holdings = data_file["Holdings"].set_index("Name")
    holdings = holdings.loc[holdings.index != "Cash EUR"]
    holdings["Portfolio Weighting %"] = (
        holdings["Portfolio Weighting %"] / holdings["Portfolio Weighting %"].sum())

    sheets = file.sheet_names
    sheets.remove("Holdings")
    transparency = {}

    for sheet in sheets:
        temp = data_file[sheet].set_index("Name").iloc[:, 1:]
        temp = temp.loc[temp.index != "Cash EUR"]
        temp = temp.loc[holdings.index]
        transparency[sheet] = temp / 100

    full_matrix = pd.DataFrame()
    for key in transparency:
        full_matrix = pd.concat([full_matrix, transparency[key]], axis=1)

    full_matrix_numpy = full_matrix.to_numpy()
    w0 = holdings["Portfolio Weighting %"].loc[full_matrix.index].to_numpy()

    drop_down_list_asset = list(full_matrix.index) + ["All"]
    drop_down_list_sector = list(full_matrix.columns)
    drop_down_list = drop_down_list_asset + drop_down_list_sector + [None]
    constraints_options = ["=", "≥", "≤"]
    bounds_sectors = {}
    
    dropdown1 = widgets.Dropdown(description='Assets:', value=None, options=drop_down_list)
    dropdown2 = widgets.Dropdown(description='Sign:', options=constraints_options)
    dropdown3 = widgets.FloatText(description='Limit')

    for col in full_matrix.columns:
        min_bounds = round(full_matrix[col].min(), 4)
        max_bounds = round(full_matrix[col].max(), 4)
        name_max = full_matrix[col].idxmax()
        name_min = full_matrix[col].idxmin()

        bounds_sectors[col] = [min_bounds, max_bounds, name_min, name_max]

    bounds_sectors_dataframe = (
        pd.DataFrame(
            bounds_sectors, index=["Lower Bound", "Upper Bound", "Name Min", "Name Max"]
        )
        .T.round(4)
        .reset_index()
        .rename(columns={"index": "Sectors"})
    )

    def on_add_constraint_clicked(b):
        row = {
            "Asset": dropdown1.value,
            "Sign": dropdown2.value,
            "Limit": dropdown3.value,
        }
        data.append(row)
        with constraint_output:
            constraint_output.clear_output()
            display(pd.DataFrame(data))

    add_constraint_btn = widgets.Button(
        description="Add Constraint", button_style="success"
    )
    add_constraint_btn.on_click(on_add_constraint_clicked)

    def on_optimize_clicked(b):

        global opt_weights, res, change,sectors

        constraint_df = pd.DataFrame(data)
        constraints = build_constraint(constraint_df.to_numpy())
        bounds = [(0, 1) for _ in range(full_matrix.shape[0])]

        if constraints:
            result = minimize(
                objective, w0, method="SLSQP", bounds=bounds, constraints=constraints
            )
        else:
            result = minimize(
                objective, w0, method="SLSQP", bounds=bounds, constraints=[]
            )

        opt_weights = result.x

        initial = pd.DataFrame(w0, index=full_matrix.index, columns=["Initial"])
        optimal = pd.DataFrame(
            opt_weights, index=full_matrix.index, columns=["Optimised"]
        )

        res = pd.concat([initial, optimal], axis=1)

        change = res.copy()

        for col in change.columns:

            change[col] = res[col] - res["Initial"]

        exposure = (res.T @ full_matrix).T.round(4)
        sectors=(res.T @ full_matrix).T.round(4)
        change = (change.T @ full_matrix).T.round(4)

        with output:
            output.clear_output()
            display(display_scrollable_df(res))
            display(display_scrollable_df(sectors))
            display(display_scrollable_df(change))

    constraint_output = widgets.Output()
    output = widgets.Output()
    optimize_btn = widgets.Button(
        description="Optimize Portfolio", button_style="primary"
    )
    optimize_btn.on_click(on_optimize_clicked)

    def on_clear_clicked(b):
        data.clear()
        with constraint_output:
            constraint_output.clear_output()
            display(
                display_scrollable_df(pd.DataFrame(columns=["Asset", "Sign", "Limit"]))
            )

    clear_btn = widgets.Button(description="Clear All", button_style="danger")
    clear_btn.on_click(on_clear_clicked)

    constraint_ui = widgets.VBox(
        [
            widgets.VBox([dropdown1, dropdown2, dropdown3]),
            widgets.HBox([add_constraint_btn, clear_btn, optimize_btn]),
            constraint_output,
        ]
    )

    centered_constraint_ui = widgets.VBox(
        [constraint_ui, output],
        layout=widgets.Layout(
            display="flex",
            justify_content="center",
            align_items="center",
            width="auto",
            padding="10px",
        ),
    )
    
    tab_contents = ['Portfolio Optimization', 'Transparency Matrix']
    
    matrix_output=widgets.Output()
    with matrix_output:
        display(display(display_scrollable_df(full_matrix)))
    
    children = [centered_constraint_ui,matrix_output]
    tab = widgets.Tab()
    tab.children = children
    
    for i, title in enumerate(tab_contents):
        tab.set_title(i, title)

    display(tab)


In [50]:
display_app(file)